In [1]:
import pandas as pd
import numpy as np
#from autoviz.AutoViz_Class import AutoViz_Class
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
appli = pd.read_csv('application_data.csv')
prev = pd.read_csv("previous_application.csv")
col = pd.read_csv('columns_description.csv')

In [3]:
appli = appli.iloc[0:50000]
prev = prev.iloc[0:50000]
#appli = appli.iloc[:, : 42]
#prev = prev.iloc[:, : 42]

In [4]:
prev.shape, appli.shape

((50000, 37), (50000, 122))

In [5]:
miss_application_data = pd.DataFrame((prev.isnull().sum())*100/prev.shape[0]).reset_index()
miss_application_data.head()

,index,0
0,SK_ID_PREV,0.000
1,SK_ID_CURR,0.000
2,NAME_CONTRACT_TYPE,0.000
3,AMT_ANNUITY,21.184
4,AMT_APPLICATION,0.000


1 - delete comme en dessous

In [6]:
prev = prev.loc[:, prev.isnull().mean() < 0.30] #supprime les cols avec 30% de valeurs manquantes
prev.shape

(50000, 26)

2 - remplacer par valeurs médianes ? 

In [7]:
vec = miss_application_data.loc[miss_application_data[0]>=30]["index"]
vec

6              AMT_DOWN_PAYMENT
12            RATE_DOWN_PAYMENT
13        RATE_INTEREST_PRIMARY
14     RATE_INTEREST_PRIVILEGED
20              NAME_TYPE_SUITE
31           DAYS_FIRST_DRAWING
32               DAYS_FIRST_DUE
33    DAYS_LAST_DUE_1ST_VERSION
34                DAYS_LAST_DUE
35             DAYS_TERMINATION
36    NFLAG_INSURED_ON_APPROVAL
Name: index, dtype: object

In [8]:
# a finir
'''
for i in vec:
    prev.loc[prev[i]==prev[i].median()]
'''

'\nfor i in vec:\n    prev.loc[prev[i]==prev[i].median()]\n'

In [9]:
appli = appli.loc[:, appli.isnull().mean() < 0.30] #supprime les cols avec 30% de valeurs manquantes
appli.shape

(50000, 72)

### rencodage selon type

In [10]:
cate = [i for i in prev.select_dtypes(include = np.object).columns if i not in ["type"] ]
num = [i for i in prev.select_dtypes(include = np.number).columns if i not in ['SK_ID_CURR'] + [ 'TARGET']]

C:\Users\Adrien\AppData\Local\Temp/ipykernel_28848/1768624195.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cate = [i for i in prev.select_dtypes(include = np.object).columns if i not in ["type"] ]


In [11]:
application_data_x = appli[[x for x in appli.columns]]
previous_application_x = prev[[x for x in prev.columns]]
application_data_x["type"] = "application_data"
previous_application_x["type"] = "previous_application"
data = pd.concat([application_data_x,previous_application_x],axis=0) 
data = data[data['TARGET'].notna()]

## a partir d'ici, plus de nan values pour les perfs du modèle

In [12]:
from sklearn.model_selection import train_test_split

y = data.TARGET
X = data.drop(['TARGET'],axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(
                                                    X, y, train_size=0.8, test_size=0.2, random_state=0)

cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)


low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

C:\Users\Adrien\anaconda3\envs\machlearn\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

### implémentation possible pour un DF qui se mettrait à jour 

In [ ]:
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

data_withid = data.reset_index() # adds an `index` column
train_set, test_set = split_train_test_by_id(data_withid, 0.2, "index")

In [14]:
X_train

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,WEEKDAY_APPR_PROCESS_START,type,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
20330,Cash loans,F,N,Y,Working,Secondary / secondary special,Married,House / apartment,MONDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17532,Cash loans,F,N,N,Working,Secondary / secondary special,Married,House / apartment,MONDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45819,Cash loans,M,Y,N,Working,Higher education,Married,House / apartment,SATURDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34807,Cash loans,M,Y,Y,Working,Higher education,Married,House / apartment,WEDNESDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31888,Cash loans,M,Y,N,State servant,Higher education,Married,House / apartment,WEDNESDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,Cash loans,M,N,Y,Working,Higher education,Married,Office apartment,SUNDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45891,Cash loans,F,N,Y,Commercial associate,Higher education,Separated,House / apartment,FRIDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42613,Cash loans,M,Y,N,Commercial associate,Incomplete higher,Married,House / apartment,SUNDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43567,Cash loans,M,Y,Y,Commercial associate,Secondary / secondary special,Married,House / apartment,FRIDAY,application_data,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [16]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from One-Hot Encoding:") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from One-Hot Encoding:
0.156579


ici, model pour selectionner les features importantes

In [43]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()

In [44]:
model.fit(OH_X_train,y_train)

C:\Users\Adrien\anaconda3\envs\machlearn\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:51:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [45]:
predict_train = model.predict(OH_X_train)
predict_train
pred = pd.DataFrame(predict_train)
rslt_df = pred[pred[0] == 1]
rslt_df

,0
60,1.0
78,1.0
272,1.0
273,1.0
309,1.0
...,...
39370,1.0
39452,1.0
39468,1.0
39790,1.0


In [46]:
trainaccuracy = accuracy_score(y_train,predict_train)
print('accuracy_score on train dataset : ', trainaccuracy)

accuracy_score on train dataset :  0.929575


In [47]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train,predict_train)

0.9952153110047847

In [48]:
predict_test = model.predict(OH_X_valid)

In [49]:
testaccuracy = accuracy_score(y_valid,predict_test)
testaccuracy

0.9193

simulation pour un individu

In [73]:
pred_util = model.predict(OH_X_valid)
y = pd.DataFrame(pred_util)

In [75]:
import random 
r = random.randint(0, len(OH_X_valid))
x = pd.DataFrame(OH_X_valid)
x.iloc[r,],y.iloc[r,]

(SK_ID_CURR                    143710.000000
 CNT_CHILDREN                       0.000000
 AMT_INCOME_TOTAL              126000.000000
 AMT_CREDIT                    292500.000000
 REGION_POPULATION_RELATIVE         0.028663
                                   ...      
 37                                 0.000000
 38                                 1.000000
 39                                 0.000000
 40                                 0.000000
 41                                 1.000000
 Name: 37737, Length: 86, dtype: float64,
 0    0.0
 Name: 1346, dtype: float64)